In [ ]:
%matplotlib widget

# LIGHT SHEET IMAGING

This is a simple notebook to control the light-sheet microscope. 
This jupyter notebook will be used to control all microscope software:
- Galvanometric mirrors
- Piezoelectric actuator
- Camera trigger*

\* For this demo we will only use Python to trigger the camera, and we will let the Hamamatsu software (HCImage) deal with the acquisition. Acquisition and saving parameters will have to be defined there.


In [ ]:
#Some necessary imports

### NI board communication
from nidaqmx import Task
from nidaqmx.constants import Edge, AcquisitionType
from nidaqmx.stream_writers import AnalogMultiChannelWriter

### PyQt imports
from PyQt5.QtWidgets import QWidget, QApplication, QSlider, QVBoxLayout, QHBoxLayout, QLabel, QPushButton, QFileDialog
from PyQt5 import QtCore
from superqt import QLabeledDoubleSlider


### Stuff
import numpy as np
import tifffile as tiff
from pathlib import Path
from scipy import fft, signal
import matplotlib.pyplot as plt
import flammkuchen as fl
import time
import sys

# First attempt

In [ ]:
#Create a task object to communicate with the NI board

###Here we need to define the NI board device and ports to communicate with, the rate at which we'll do it, and the type of signal we want to send.
task = Task()
task.ao_channels.add_ao_voltage_chan("Dev1/ao0:3", min_val=-10, max_val=10) 
task.timing.cfg_samp_clk_timing(
    rate=40000,
    source="OnboardClock",
    active_edge=Edge.RISING,
    sample_mode=AcquisitionType.CONTINUOUS,
    samps_per_chan=10000)

In [ ]:
#Now we need to create the arrays with the signals that we will send to the different microscope elements
t = np.linspace(0, 1, 10000)

### XY SCANNING
### Fast scanning of the fast galvo to generate a planar sheet of light 
xy_signal = signal.sawtooth(2 * np.pi * 100 * t, width=0.5) 

### CAMERA TRIGGER 
### Single pulse at the start of the task to trigger the camera acquisition
cam_signal = np.full_like(xy_signal, 0)
cam_signal[0] = 5
    
### Z SCANNING
### Slower scanning of the slow galvo to move the light sheet up and down
z_signal = signal.sawtooth(2 * np.pi * 1 * t)

### PIEZO 
### Piezo motion. Should be synchronized with the z_signal to keep the light sheet at the focal plane of the objective
piezo_signal = signal.sawtooth(2 * np.pi * 1 * t, width=0)

In [ ]:
#This is how the signals will look like
plt.figure(figsize=(5, 6))

for i, (each_signal, label) in enumerate(zip([xy_signal, z_signal, piezo_signal, cam_signal], ['xy_signal', 'z_signal', 'piezo_signal', 'cam_signal'])): 
    plt.plot(each_signal+i*5, label=label)

plt.legend(bbox_to_anchor=(1.15, 1.05))
plt.tight_layout()

In [ ]:
#Concatenate all signals onto a single array to send to the NI board
signal_array = np.stack([piezo_signal, 
                         z_signal, 
                         xy_signal, 
                         cam_signal]) #order matters

If everything is ready, running the following chunk will trigger the microscope acquisition. Make sure everything is ready on the side of the HCImage software.

In [ ]:
#Start acquisition
task.write(signal_array, auto_start=True)

In [ ]:
#End acquisition
task.stop()

# Calibration

This first attempt probably did not work out. Can you guess what is missing?

The following code will start a GUI to deal with that. If it does not run, it probably is because the NI board is stuck dealing with the previous signals. Restart the python kernel and start running the code form here.

In [ ]:
class CalibrationWidget(QWidget):
    def __init__(self):
        super().__init__()

        self.dev = "Dev1"
        self.ao_zgalvo = "ao1"# fast galvo
        self.ao_piezo = "ao0" # slow galvo
        self.ao_scangalvo = "ao2"
        
        self.zgalvo_pos = 0
        self.zgalvo_range = [-1.5, 1.5]
        
        self.piezo_pos = 0
        self.piezo_range = [-2, 10]
        self.set_tasks()
        self.scan_galvo()
        
        self.calibration_points = []
        
        self.set_gui()


    def set_gui(self):
        self.zgalvo_slider = QLabeledDoubleSlider(QtCore.Qt.Horizontal)
        self.zgalvo_slider.setRange(self.zgalvo_range[0], self.zgalvo_range[1])
        self.zgalvo_slider.setValue(0)
        self.zgalvo_slider.valueChanged.connect(self.update)
        zgalvo_layout = QHBoxLayout()
        zgalvo_layout.addWidget(QLabel("Z galvo pos."))
        zgalvo_layout.addWidget(self.zgalvo_slider)
        
        
        self.piezo_slider = QLabeledDoubleSlider(QtCore.Qt.Horizontal)
        self.piezo_slider.setRange(self.piezo_range[0], self.piezo_range[1])
        self.piezo_slider.setValue(0)
        self.piezo_slider.valueChanged.connect(self.update)
        piezo_layout = QHBoxLayout()
        piezo_layout.addWidget(QLabel("Piezo pos."))
        piezo_layout.addWidget(self.piezo_slider)
        
        self.save_button = QPushButton("Store calibration point")
        self.save_button.clicked.connect(self.save_values)
        self.remove_button = QPushButton("Remove last calibration point")
        self.remove_button.clicked.connect(self.remove_values)
        saving_layout = QHBoxLayout()
        saving_layout.addWidget(self.save_button)
        saving_layout.addWidget(self.remove_button)
        
        self.points_label = QLabel("No calibration points yet")
        self.points_label.setWordWrap(True)
        
        self.save_button = QPushButton("Save calibration")
        self.save_button.clicked.connect(self.save_calibration)
        self.save_button.setEnabled(False)

        layout = QVBoxLayout()
        layout.addLayout(zgalvo_layout)
        layout.addLayout(piezo_layout)
        layout.addLayout(saving_layout)
        layout.addWidget(self.points_label)
        layout.addWidget(self.save_button)
        
        self.setLayout(layout)
        
    def save_calibration(self):
        filename = QFileDialog.getSaveFileName(filter="hdf5 files (*.h5)")[0]
        saved_data = {'metadata':{'order':["galvo", "piezo"],
                                  'ranges':[self.zgalvo_range, self.piezo_range],
                                     },
                      'cal_points': self.calibration_points
                     }
        fl.save(filename, saved_data)
    
    def save_values(self):
        self.calibration_points.append([self.zgalvo_pos, self.piezo_pos])
        self.print_points()
        
    def remove_values(self):
        self.calibration_points.pop(-1)
        self.print_points()
        
    def print_points(self):
        if len(self.calibration_points) == 0:
            self.points_label.setText("No calibration points yet")
        else:
            label_text = "{} calibration points:".format(len(self.calibration_points))
            
            for point in self.calibration_points:
                label_text = label_text + "\nZ galvo pos.: {:.3f}, piezo pos.: {:.3f}".format(point[0], point[1])
            self.points_label.setText(label_text)
            
        if len(self.calibration_points) >= 3:
            self.save_button.setEnabled(True)
        else:
            self.save_button.setEnabled(False)
        

    def set_tasks(self):
        self.cal_task = Task()
        self.cal_task.ao_channels.add_ao_voltage_chan(f"{self.dev}/{self.ao_zgalvo}", min_val=self.zgalvo_range[0], max_val=self.zgalvo_range[1])
        self.cal_task.ao_channels.add_ao_voltage_chan(f"{self.dev}/{self.ao_piezo}", min_val=self.piezo_range[0], max_val=self.piezo_range[1])
        # self.cal_task.timing.cfg_samp_clk_timing(
        #     rate=1000,
        #     source="OnboardClock",
        #     active_edge=Edge.RISING,
        #     sample_mode=AcquisitionType.FINITE,
        #     samps_per_chan=1000
        # )
        
        self.scan_task = Task()
        self.scan_task.ao_channels.add_ao_voltage_chan(f"{self.dev}/{self.ao_scangalvo}", min_val=-10, max_val=10)
        self.scan_task.timing.cfg_samp_clk_timing(
            rate=40000,
            source="OnboardClock",
            active_edge=Edge.RISING,
            sample_mode=AcquisitionType.CONTINUOUS,
            samps_per_chan=10000)

    
    def scan_galvo(self):
        t = np.linspace(0, 1, 10000)
        xy_signal = signal.sawtooth(2 * np.pi * 100 * t, width=.5)*2
        self.scan_task.write(xy_signal, auto_start=True)

    def update(self):

        self.zgalvo_pos = self.zgalvo_slider.value()
        self.piezo_pos = self.piezo_slider.value()
        
        self.cal_task.write(np.array([self.zgalvo_pos, 
                                      self.piezo_pos]), auto_start=True)
        

if __name__ == '__main__':
    app = QApplication(sys.argv)
    main = CalibrationWidget()
    main.show()
    sys.exit(app.exec_())


In [ ]:
if __name__ == '__main__':
    app = QApplication(sys.argv)
    main = CalibrationWidget()
    main.show()
    sys.exit(app.exec_())

# Re-try acquisition

In [ ]:
calibration_file = fl.load(Path(r"C:\Users\Admin\Desktop\cal.h5"))

galvo_range = calibration_file['metadata']['ranges'][0]
piezo_range = calibration_file['metadata']['ranges'][1]

In [ ]:
calibration_file

In [ ]:
acq_task = Task()
acq_task.ao_channels.add_ao_voltage_chan("Dev1/ao1", min_val=galvo_range[0], max_val=galvo_range[1]) #Z galvo
acq_task.ao_channels.add_ao_voltage_chan("Dev1/ao2", min_val=-10, max_val=10) #XY galvo
acq_task.ao_channels.add_ao_voltage_chan("Dev1/ao0", min_val=piezo_range[0], max_val=piezo_range[1]) #Piezo
acq_task.ao_channels.add_ao_voltage_chan("Dev1/ao3", min_val=-10, max_val=10) #Camera

acq_task.timing.cfg_samp_clk_timing(
    rate=30000,
    source="OnboardClock",
    active_edge=Edge.RISING,
    sample_mode=AcquisitionType.CONTINUOUS,
    # samps_per_chan=10000,
)

In [ ]:
#Load calibration points
calibration_points = calibration_file['cal_points']
galvo_cal_pts = np.array([cal[0] for cal in calibration_points])
piezo_cal_pts = np.array([cal[1] for cal in calibration_points])

#Sort calibration points 
ordered_pts = np.argsort(galvo_cal_pts)
galvo_cal_pts = galvo_cal_pts[ordered_pts]
piezo_cal_pts = piezo_cal_pts[ordered_pts]

# Fit line
linear_model = np.polyfit(galvo_cal_pts, piezo_cal_pts, 1)
linear_model_fn = np.poly1d(linear_model)

#Define scanning range (based on Z galvo values)
galvo_scan_range = [-.22, 0] #[0.0096, 0.06]

#Generate arrays to write on NI boards
t = np.linspace(0, 1, 10000)

z_galvo_arr = np.linspace(galvo_scan_range[0], galvo_scan_range[1], t.shape[0])

piezo_arr = linear_model_fn(z_galvo_arr)

camera_arr = np.full_like(t, 5)
# camera_arr[0] = 5

xy_galvo_arr = signal.sawtooth(2 * np.pi * 100 * t, width=0.5) *2

acq_task_arr = np.stack([z_galvo_arr, xy_galvo_arr, piezo_arr, camera_arr])

In [ ]:
acq_task.write(acq_task_arr, auto_start=True)

In [ ]:
acq_task.stop()